In [2]:
import pandas as pd

data = [[1, '2019-02-17', '2019-02-28', 5], [1, '2019-03-01', '2019-03-22', 20], [2, '2019-02-01', '2019-02-20', 15], [2, '2019-02-21', '2019-03-31', 30]]
prices = pd.DataFrame(data, columns=['product_id', 'start_date', 'end_date', 'price']).astype({'product_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]', 'price':'Int64'})
data = [[1, '2019-02-25', 100], [1, '2019-03-01', 15], [2, '2019-02-10', 200], [2, '2019-03-22', 30]]
units_sold = pd.DataFrame(data, columns=['product_id', 'purchase_date', 'units']).astype({'product_id':'Int64', 'purchase_date':'datetime64[ns]', 'units':'Int64'})

In [3]:
df_merge = pd.merge(units_sold, prices, on="product_id", how="left")

In [4]:
df_merge.head()

,product_id,purchase_date,units,start_date,end_date,price
0,1,2019-02-25,100,2019-02-17,2019-02-28,5
1,1,2019-02-25,100,2019-03-01,2019-03-22,20
2,1,2019-03-01,15,2019-02-17,2019-02-28,5
3,1,2019-03-01,15,2019-03-01,2019-03-22,20
4,2,2019-02-10,200,2019-02-01,2019-02-20,15


In [9]:
df = df_merge[(df_merge["start_date"] <= df_merge["purchase_date"]) & (df_merge["purchase_date"] <= df_merge["end_date"] )]

In [10]:
df.head()

,product_id,purchase_date,units,start_date,end_date,price
0,1,2019-02-25,100,2019-02-17,2019-02-28,5
3,1,2019-03-01,15,2019-03-01,2019-03-22,20
4,2,2019-02-10,200,2019-02-01,2019-02-20,15
7,2,2019-03-22,30,2019-02-21,2019-03-31,30


In [12]:
df["total_price"] = df["price"] * df["units"]

/tmp/ipykernel_10132/2495675439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["total_price"] = df["price"] * df["units"]


In [13]:
df.head()

,product_id,purchase_date,units,start_date,end_date,price,total_price
0,1,2019-02-25,100,2019-02-17,2019-02-28,5,500
3,1,2019-03-01,15,2019-03-01,2019-03-22,20,300
4,2,2019-02-10,200,2019-02-01,2019-02-20,15,3000
7,2,2019-03-22,30,2019-02-21,2019-03-31,30,900


In [15]:
df2 = df.groupby("product_id").agg({"units": "sum", "total_price": "sum"}).reset_index()
df2.head()

,product_id,units,total_price
0,1,115,800
1,2,230,3900


In [16]:
df2["avg"] = df2["total_price"] / df2["units"]

In [19]:
df2.round(3)

,product_id,units,total_price,avg
0,1,115,800,6.957
1,2,230,3900,16.957


In [ ]:
import pandas as pd

def average_selling_price(prices: pd.DataFrame, units_sold: pd.DataFrame) -> pd.DataFrame:
    df_merge = pd.merge(units_sold, prices, on="product_id", how="left")
    df = df_merge[(df_merge["start_date"] <= df_merge["purchase_date"]) & (df_merge["purchase_date"] <= df_merge["end_date"] )]
    df["total_price"] = df["price"] * df["units"]
    df2 = df.groupby("product_id").agg({"units": "sum", "total_price": "sum"}).reset_index()
    df2["average_price"] = df2["total_price"] / df2["units"]
    df2 = df2.round(2)
    df3 = df2[["product_id", "average_price"]]
    df4 = pd.merge(df3, prices["product_id"].drop_duplicates(), on="product_id", how="right")
    df4["average_price"] = df4["average_price"].fillna(0)
    df5 = df4[["product_id", "average_price"]]
    return df5

In [ ]:
mysql = """

# Write your MySQL query statement below
WITH product_list AS (
    SELECT distinct product_id 
    FROM prices
), average_price AS (
SELECT s.product_id, COALESCE(ROUND(SUM(s.units*p.price)/SUM(s.units), 2) , 0) AS average_price
-- p.start_date, p.end_date, s.purchase_date, p.price, s.units
FROM prices p
LEFT JOIN unitssold s ON p.product_id = s.product_id
RIGHT JOIN product_list pl ON pl.product_id = s.product_id
WHERE s.purchase_date >= p.start_date
AND s.purchase_date <= p.end_date
GROUP BY product_id 
) 
SELECT pl.product_id, COALESCE(average_price, 0) AS average_price
FROM product_list pl
LEFT JOIN average_price ap ON pl.product_id = ap.product_id
"""